# Introduction

combination of r_translation.py script and more_r_translation.py marimo notebook. these olds scripts are under `old_files`

for the second matching wrangle, the parts for specificEpithet and infraSpecificEpithet doesn't match with many results.

# Imports and Initial Preprocessing

In [ ]:
import marimo as mo
import polars as pl

In [ ]:
bos = (
    pl.read_csv("bos.csv")
    .rename(
        {
            "data cleanup changes": "data cleanup changes 1",
            "species": "specificEpithet",
            "subspecies": "infraspecificEpithet",
            "domain ": "domain",
        }
    )
    .with_columns(pl.col(pl.String).str.strip_chars(" "))
    .with_columns(genericName=pl.col("genus"))
)

In [ ]:
null_counts = (
    bos.null_count()
    .transpose(include_header=True)
    .rename({"column": "feature", "column_0": "len"})
)
columns_to_drop_as_all_nulls = (
    null_counts.filter(null_counts["len"] == bos.shape[0])
    .transpose()[0, :]
    .transpose()
    .to_series()
    .to_list()
)

In [ ]:
columns_to_drop_as_they_are_just_changes_logs = [
    "cleanup changes ",
    "Data cleanup changes",
    "data cleanup changes 1",
    "changes",
    "cleanup changes",
    "Unnamed: 18",
    "Unnamed: 17",
    "Unnamed: 19",
    "query",
    "query ",
    "issue",
    "cleanup changes/comments",
    "subphylum",
]

In [ ]:
bos_cleaned = (
    (
        bos.drop(columns_to_drop_as_all_nulls)
        .drop(columns_to_drop_as_they_are_just_changes_logs)
        .select(~pl.selectors.starts_with("Unnamed"))
    )
    .with_columns(
        infraspecificEpithet=pl.when(pl.col("infraspecificEpithet").is_null())
        .then(pl.lit(""))
        .otherwise(pl.col("infraspecificEpithet"))
    )
    .with_columns(
        infraspecificEpithet=pl.when(
            (pl.col("genericName") == "Glenea")
            & (pl.col("specificEpithet") == "mathemathica")
        )
        .then(pl.lit("mathematica"))
        .otherwise(pl.col("infraspecificEpithet"))
    )
    .with_columns(
        specificEpithet=pl.when(
            (pl.col("genericName") == "Glenea")
            & (pl.col("specificEpithet") == "mathemathica")
        )
        .then(pl.lit("mathematica"))
        .when(
            (pl.col("genericName") == "Bavia")
            & (pl.col("specificEpithet") == "sexupunctata")
        )
        .then(pl.lit("sexpunctata"))
        .when(
            (pl.col("genericName") == "Omoedus")
            & (pl.col("specificEpithet") == "ephippigera")
        )
        .then(pl.lit("ephippiger"))
        .when(
            (pl.col("genericName") == "Byblis")
            & (pl.col("specificEpithet") == "kallartha")
        )
        .then(pl.lit("kallarthra"))
        .when(
            (pl.col("genericName") == "Alcockpenaeopsis")
            & (pl.col("specificEpithet") == "hungerfordi")
        )
        .then(pl.lit("hungerfordii"))
        .when(
            (pl.col("genericName") == "Pseudosesarma")
            & (pl.col("specificEpithet") == "edwardsi")
        )
        .then(pl.lit("edwardsii"))
        .when(
            (pl.col("genericName") == "Urocaridella")
            & (pl.col("specificEpithet") == "antonbruuni")
        )
        .then(pl.lit("antonbruunii"))
        .when(
            (pl.col("genericName") == "Ocypode")
            & (pl.col("specificEpithet") == "cordimanus")
        )
        .then(pl.lit("cordimana"))
        .otherwise(pl.col("specificEpithet"))
    )
    .with_columns(
        domain=pl.lit("Eukarya"),
        kingdom=pl.when(
            (pl.col("phylum").is_in(["Ascomycota", "Basidiomycota"]))
        )
        .then(pl.lit("fungi"))
        .when(pl.col("phylum") == "Cyanobacteria")
        .then(pl.lit("Bacteria"))
        .when(pl.col("phylum") == "Ciliophora")
        .then(pl.lit("Protista"))
        .otherwise(pl.lit("Animalia")),
    )
    .with_columns(
        taxonName=pl.col("taxonName")
        .str.replace_all("<i>", "")
        .str.replace_all("</i>", ""),
    )
)

In [ ]:
_filter = pl.col("taxonomicStatus").is_in(
    [
        "accepted",
        "synonym",
        "homotypic synonym",
        "proparte synonym",
        "heterotypic synonym",
    ]
)
taxon_ranked_only = (
    pl.scan_csv(
        "gbif/Taxon.tsv",
        separator="\t",
        quote_char=None,
        cache=True,
    )
    .filter((pl.col("taxonRank") != "unranked") & _filter)
    .select(
        "taxonID",
        "genericName",
        "genus",
        "specificEpithet",
        "infraspecificEpithet",
        "taxonomicStatus",
        "acceptedNameUsageID",
        # "domain",
    )
    .filter((pl.col("genus") != "") & (pl.col("specificEpithet") != ""))
    .with_columns(
        infraspecificEpithet=pl.when(
            (pl.col("infraspecificEpithet").is_null())
        )
        .then(pl.lit(""))
        .otherwise(pl.col("infraspecificEpithet"))
    )
)
taxon_ranked_only = taxon_ranked_only.with_columns(
    genus=pl.when(pl.col("genus").is_null())
    .then("genericName")
    .otherwise("genus")
)

In [ ]:
matching = (
    taxon_ranked_only.join(
        other=pl.LazyFrame(bos_cleaned),
        on=["genus", "specificEpithet", "infraspecificEpithet"],
        how="right",
    )
    .rename({"taxonID": "matched_taxonID"})
    .select(
        "speciesId",
        "matched_taxonID",
        "acceptedNameUsageID",
        "taxonName",
        "domain",
        "kingdom",
        "phylum",
        "class",
        "subclass",
        "superorder",
        "order",
        "sub-order",
        "infraorder",
        "section",
        "subsection",
        "superfamily",
        "family",
        "subfamily",
        "tribe",
        "genus",
        "subgenus",
        "genericName",
        "specificEpithet",
        "infraspecificEpithet",
    )
    .with_columns(
        pl.when(pl.col("infraspecificEpithet") == "")
        .then(pl.lit(None))
        .otherwise("infraspecificEpithet")
        .alias("infraspecificEpithet")
    )
)
# matching and contetnious split
# What makes a data point contentious is where it has duplicate speciesId.
matching = matching.filter(~pl.col("speciesId").is_duplicated()).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(None))
    .cast(pl.Int64)
)
contentious = matching.filter(
    (pl.col("speciesId").is_duplicated())
).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(None))
    .cast(pl.Int64)
)

unique_contentious = contentious.filter(
    (pl.col("acceptedNameUsageID").is_null())
    & (
        pl.col("matched_taxonID").is_in(
            pl.col("acceptedNameUsageID").implode()
        )
    )
)

unique_contentious_speciesId = (
    unique_contentious.select("speciesId").collect().to_series().implode()
)  # Just the speciesIds
contentious = contentious.filter(
    ~pl.col("speciesId").is_in(unique_contentious_speciesId)
)  # Removing unique conentious

matching = pl.concat(
    [matching, unique_contentious],
)

matching = matching.with_columns(
    genus=pl.when(
        (pl.col("genus").is_null()) & (pl.col("specificEpithet").is_not_null())
    )
    .then(pl.col("taxonName").str.split(" ").list[0])
    .otherwise("genus")
)

no_match = (
    matching.filter(pl.col("matched_taxonID").is_null())
    .with_columns(
        taxonRank=pl.lit("BOSuncornirmedSpecies"),
        taxonomicStatus=pl.lit("BOSunformired"),
        parentNameUsageID=pl.lit(None),
    )
    .fill_null("")
    .collect()
)

# First Matching Wrangle

In [ ]:
repeated_accepted_taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(pl.col("taxonomicStatus") == pl.lit("accepted"))
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .filter(~pl.col("canonicalName").is_null())
    .filter(pl.col("canonicalName") != "")
    .sort("canonicalName")
    .filter(pl.col("canonicalName").is_duplicated())
)

In [ ]:
priority_columns = [
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
]

In [ ]:
_schema = {
    "feature_that_is_equal_to_canonicalName": pl.String,
    "matches": pl.String,
}
collected_repeated_taxons = repeated_accepted_taxons.collect()
RAT_interim = pl.DataFrame(schema=_schema)
for _col in priority_columns:
    _canonical_names = (
        collected_repeated_taxons.filter(
            pl.col("canonicalName") == pl.col(_col)
        )
        .select("canonicalName")
        .unique()
        .to_series()
        .to_list()
    )
    if len(_canonical_names) != 0:
        _row = pl.DataFrame(
            data={
                "feature_that_is_equal_to_canonicalName": _col,
                "matches": _canonical_names,
            },
            schema=_schema,
        )
        RAT_interim = RAT_interim.vstack(_row)

RAT_feats = (
    RAT_interim.group_by("matches")
    .agg(pl.col("feature_that_is_equal_to_canonicalName").str.join(", "))
    .with_columns(
        pl.col("feature_that_is_equal_to_canonicalName").str.split(", ")
    )
    .sort("feature_that_is_equal_to_canonicalName")
)
RAT_feats

matches,feature_that_is_equal_to_canonicalName
str,list[str]
"""Micrognathozoa""","[""class"", ""phylum""]"
"""Passeriformes""","[""family"", ""order""]"
"""Sirenia""","[""family"", ""order""]"
"""Hildebrandtiella""","[""genus""]"
"""Bartramia""","[""genus""]"
…,…
"""Diplura""","[""genus"", ""order"", ""class""]"
"""Acanthocephala""","[""genus"", ""phylum""]"
"""Ctenophora""","[""genus"", ""phylum""]"


In [ ]:
updated_to_matching = []
still_no_match = []
_collected_repeated_taxons = collected_repeated_taxons.fill_null("")
_reversed_priority_columns = priority_columns.copy()
_reversed_priority_columns.reverse()
pl.Config.set_tbl_cols(10)
for _col in _reversed_priority_columns:
    for _match in RAT_feats["matches"]:
        # skipping those that are not in RAT_feats
        if _match not in no_match[_col].to_list():
            continue

        taxon_data_to_select_from = _collected_repeated_taxons.filter(
            pl.col("canonicalName") == _match
        ).select(["taxonID"] + _reversed_priority_columns)

        # Select from the two rows what taxon data to select.
        selected_row_int = -1
        for i, _tuple in enumerate(
            taxon_data_to_select_from.select(
                ["taxonID"] + priority_columns
            ).iter_rows()
        ):
            col_index = priority_columns.index(_col)
            _x = _tuple[col_index]
            if not bool(_x):
                selected_row_int = i  # Since there are usually only two rows from taxon_data_to_select_from, the results is either 0 or 1
                break
        assert selected_row_int != -1

        # has_predicament1 is when kingdom to genus all has values and it is kind of leveled between the two rows of the taxon data.
        has_predicament1 = False
        _l = taxon_data_to_select_from["family"].to_list()
        if _col == "genus" and _l[0] is not None and _l[1] is not None:
            has_predicament1 = True

        chosen_taxonId = taxon_data_to_select_from[selected_row_int, 0]
        other_taxonId = taxon_data_to_select_from[
            int(
                not bool(selected_row_int)
            ),  # converts the int value into boolean than reverse the boolean before converting back to int.
            0,
        ]

        # Getting the _no_match_subset_to_update section
        _no_match_subset_to_update = no_match.filter(
            pl.col(_col) == _match,
        )
        for _col3 in priority_columns[: priority_columns.index(_col)]:
            _no_match_subset_to_update = _no_match_subset_to_update.filter(
                pl.col(_col3) == "",
            )
        if not _no_match_subset_to_update.select(priority_columns).is_empty():
            pass
        else:
            continue
        _x = _reversed_priority_columns[:-3]
        feature_to_find_nulls = (
            _x[_reversed_priority_columns.index(_col) + 1]
            if len(_x) > _reversed_priority_columns.index(_col) + 1
            else None
        )  # just the next feature to find null
        if feature_to_find_nulls is not None:
            _no_match_subset_to_update = _no_match_subset_to_update.filter(
                pl.col(feature_to_find_nulls) == ""
            )
            # print(_no_match_subset_to_update)
            # if _no_match_subset_to_update.is_empty():
            #     still_no_match_subset = no_match.filter(
            #         pl.col(_col) == _match, # This is the problme where there is too little matches in second matching wrangling
            #     ).with_columns(
            #         current_feature=pl.lit(_col), current_name=pl.lit(_match)
            #     )
            #     still_no_match.append(still_no_match_subset)
            #     continue

        # Settling predicament one
        # this predicament thing should only happen when the subset is of only 1 row.
        # has_predicament1 is when kingdom to genus all has values and it is kind of leveled between the two rows of the taxon data.
        if has_predicament1 and _no_match_subset_to_update.shape[0] == 1:
            match = True
            for _col1 in [
                "class",
                "order",
                "family",
                "genus",
                "specificEpithet",
                "infraspecificEpithet",
            ]:
                match *= (
                    _no_match_subset_to_update[_col1].item()
                    == taxon_data_to_select_from[selected_row_int, :][
                        _col1
                    ].item()
                )
            if not match:
                match = True
                for _col1 in [
                    "class",
                    "order",
                    "family",
                    "genus",
                    "specificEpithet",
                    "infraspecificEpithet",
                ]:
                    match *= (
                        _no_match_subset_to_update[_col1].item()
                        == taxon_data_to_select_from[
                            int(not bool(selected_row_int)), :
                        ][_col1].item()
                    )  # searching in the three columns of interests for matches

                # assert (
                #     match
                # )  # Double check that there is no unmatch.  # noqa: E712
                if match:
                    temp = chosen_taxonId
                    chosen_taxonId = other_taxonId
                    other_taxonId = temp

        # turning these rows to matching by filling parentNameUsageID
        _no_match_subset_to_update = _no_match_subset_to_update.with_columns(
            parentNameUsageID=pl.when(
                (pl.col("infraspecificEpithet").is_null())
                & (pl.col("specificEpithet").is_not_null())
                & (pl.col("genus").is_not_null())
            )
            .then(pl.lit(other_taxonId))
            .otherwise(pl.lit(chosen_taxonId))
        )

        updated_to_matching.append(_no_match_subset_to_update)

# turn them into dataframes and writing to csv files
updated_to_matching = pl.concat(
    updated_to_matching, rechunk=True, parallel=True
)
updated_to_matching.write_csv("first_matches_set_from_wrangling.csv")

still_no_match = no_match.join(
    updated_to_matching.select("speciesId"), on="speciesId", how="anti"
)

In [ ]:
updated_to_matching.select(["speciesId"] + priority_columns).filter(
    pl.col("genus") == "Aonides"
)

speciesId,infraspecificEpithet,specificEpithet,genus,family,order,class,phylum,kingdom
str,str,str,str,str,str,str,str,str


In [ ]:
updated_to_matching.shape

In [ ]:
still_no_match

speciesId,matched_taxonID,acceptedNameUsageID,taxonName,domain,…,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,parentNameUsageID
str,i64,i64,str,str,…,str,str,str,str,null
"""A-Anne-Clit-Hirudinea-000001""",null,null,"""Glossiphoniidae sp. ""1""""","""Eukarya""",…,"""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000002""",null,null,"""Glossiphoniidae sp. ""2""""","""Eukarya""",…,"""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000003""",null,null,"""Glossiphoniidae sp. ""3""""","""Eukarya""",…,"""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000004""",null,null,"""Hirudinidae sp. ""1""""","""Eukarya""",…,"""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000005""",null,null,"""Hirudinidae sp. ""2""""","""Eukarya""",…,"""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
…,…,…,…,…,…,…,…,…,…,…
"""A-Hemi-Enteropneusta-000010""",null,null,"""Acorn worm sp.""","""Eukarya""",…,"""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Hemi-Enteropneusta-000011""",null,null,"""Acorn worm sp.""","""Eukarya""",…,"""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Phoronida-000002""",null,null,"""Phoronida sp.""","""Eukarya""",…,"""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null


# Second Matching Wrangle

In [ ]:
taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(
        pl.col("taxonomicStatus") == pl.lit("accepted"),
        pl.col("taxonRank") != "unranked",
    )
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .select(["taxonID"] + priority_columns)
)


def join_and_parentId_insertion(
    match_on: pl.DataFrame, _taxons_subset: pl.DataFrame, on: list
) -> pl.DataFrame:
    return (
        match_on.join(_taxons_subset, on=on)
        .with_columns(
            parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
            .then("taxonID")
            .otherwise("parentNameUsageID")
        )
        .drop("taxonID")
    )

##  matching for datapoints where phylumn to class ranks features are not empty strings

In [ ]:
_filter_class = (
    pl.col("class") != "",
    pl.col("order") == "",
    pl.col("family") == "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
)
match_on_class = still_no_match.with_columns(
    name_to_match=pl.when(_filter_class).then("class").otherwise(None)
).filter(_filter_class)

_x_class = (
    still_no_match.filter(_filter_class)
    .select("class")
    .group_by("class")
    .len()
)

_taxons_subset_class = (
    taxons.filter(
        pl.col("family").is_null(),
        pl.col("order").is_null(),
        pl.col("genus").is_null(),
        pl.col("class").is_in(_x_class["class"].unique().to_list()),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
    )
    .select("taxonID", "class")
    .collect()
)
match_on_class = join_and_parentId_insertion(
    match_on_class, _taxons_subset_class, ["class"]
)

In [ ]:
match_on_class.select(["parentNameUsageID", "class"]).sort("parentNameUsageID")

parentNameUsageID,class
i64,str
136,"""Cephalopoda"""
136,"""Cephalopoda"""
136,"""Cephalopoda"""
136,"""Cephalopoda"""
137,"""Bivalvia"""
…,…
11545536,"""Copepoda"""
11545536,"""Copepoda"""
11545536,"""Copepoda"""


In [ ]:
match_on_class.select(
    ["speciesId", "taxonName", "parentNameUsageID"] + priority_columns
).sort("parentNameUsageID").filter(pl.col("parentNameUsageID") == 136)

speciesId,taxonName,parentNameUsageID,infraspecificEpithet,specificEpithet,…,family,order,class,phylum,kingdom
str,str,i64,str,str,…,str,str,str,str,str
"""A-Moll-Cephalopoda-000012""","""Squid eggs""",136,"""""","""""",…,"""""","""""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000062""","""Squid eggs sp 2""",136,"""""","""""",…,"""""","""""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000021""","""Squid sp. 1""",136,"""""","""""",…,"""""","""""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000032""","""Squid sp. 2""",136,"""""","""""",…,"""""","""""","""Cephalopoda""","""Mollusca""","""Animalia"""


## matching for datapoints where phylum to order ranks features are not empty strings

In [ ]:
_filter_order = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") == "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
)
match_on_order = still_no_match.with_columns(
    name_to_match=pl.when(_filter_order).then("order").otherwise(None)
).filter(_filter_order)
_x_order = (
    still_no_match.filter(_filter_order)
    .select("order")
    .group_by("order")
    .len()
)

_taxons_subset_order = (
    taxons.filter(
        pl.col("family").is_null(),
        pl.col("order").is_in(_x_order["order"].unique().to_list()),
        pl.col("genus").is_null(),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
    )
    .select("taxonID", "order")
    .collect()
)
match_on_order = join_and_parentId_insertion(
    match_on_order, _taxons_subset_order, ["order"]
)

In [ ]:
match_on_order.select(["parentNameUsageID", "order"]).sort("parentNameUsageID")

parentNameUsageID,order
i64,str
459,"""Octopoda"""
459,"""Octopoda"""
459,"""Octopoda"""
459,"""Octopoda"""
459,"""Octopoda"""
…,…
12202578,"""Scleralcyonacea"""
12202578,"""Scleralcyonacea"""
12202578,"""Scleralcyonacea"""


In [ ]:
match_on_order.select(
    ["speciesId", "taxonName", "parentNameUsageID"] + priority_columns
).sort("parentNameUsageID").filter(pl.col("parentNameUsageID") == 459)

speciesId,taxonName,parentNameUsageID,infraspecificEpithet,specificEpithet,…,family,order,class,phylum,kingdom
str,str,i64,str,str,…,str,str,str,str,str
"""A-Moll-Cephalopoda-000064""","""Octopus sp.""",459,"""""","""""",…,"""""","""Octopoda""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000065""","""Octopus sp.""",459,"""""","""""",…,"""""","""Octopoda""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000066""","""Octopus sp.""",459,"""""","""""",…,"""""","""Octopoda""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000067""","""Octopus sp.""",459,"""""","""""",…,"""""","""Octopoda""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000068""","""Octopus sp.""",459,"""""","""""",…,"""""","""Octopoda""","""Cephalopoda""","""Mollusca""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""A-Moll-Cephalopoda-000041""","""Octopus sp. 6""",459,"""""","""""",…,"""""","""Octopoda""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000042""","""Octopus sp. 7""",459,"""""","""""",…,"""""","""Octopoda""","""Cephalopoda""","""Mollusca""","""Animalia"""
"""A-Moll-Cephalopoda-000043""","""Octopus sp. 8""",459,"""""","""""",…,"""""","""Octopoda""","""Cephalopoda""","""Mollusca""","""Animalia"""


## matching for datapoints where phylum to family ranks features are not empty

In [ ]:
_filter_family = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
)
match_on_family = still_no_match.with_columns(
    name_to_match=pl.when(_filter_family).then("family").otherwise(None)
).filter(_filter_family)
_x_family = (
    still_no_match.filter(_filter_family)
    .select("family", "order")
    .group_by("family", "order")
    .len()
)

_taxons_subset_family = (
    taxons.filter(
        pl.col("order").is_in(_x_family["order"].unique().to_list()),
        pl.col("family").is_in(_x_family["family"].unique().to_list()),
        pl.col("genus").is_null(),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
    )
    .select("taxonID", "family")
    .collect()
)
match_on_family = join_and_parentId_insertion(
    match_on_family, _taxons_subset_family, ["family"]
)

In [ ]:
match_on_family.select(
    ["speciesId", "taxonName", "parentNameUsageID"] + priority_columns
).sort("parentNameUsageID")

speciesId,taxonName,parentNameUsageID,infraspecificEpithet,specificEpithet,…,family,order,class,phylum,kingdom
str,str,i64,str,str,…,str,str,str,str,str
"""A-Arth-Crus-Isopoda-000023""","""Sphaeromatidae sp. 1""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000024""","""Sphaeromatidae sp. 2""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000025""","""Sphaeromatidae sp. 3""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000026""","""Sphaeromatidae sp. 4""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000027""","""Sphaeromatidae sp. 5""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""A-Arth-Hexa-Diptera-002917""","""Sphaeroceridae sp. ZRC_BDP0294…",11928281,"""""","""""",…,"""Sphaeroceridae""","""Diptera""","""Insecta""","""Arthopoda""","""Animalia"""
"""A-Arth-Hexa-Diptera-002918""","""Sphaeroceridae sp. ZRC_BDP0303…",11928281,"""""","""""",…,"""Sphaeroceridae""","""Diptera""","""Insecta""","""Arthopoda""","""Animalia"""
"""A-Arth-Hexa-Diptera-002919""","""Sphaeroceridae sp. ZRC_BDP0309…",11928281,"""""","""""",…,"""Sphaeroceridae""","""Diptera""","""Insecta""","""Arthopoda""","""Animalia"""


In [ ]:
match_on_family.select(
    ["speciesId", "taxonName", "parentNameUsageID"] + priority_columns
).sort("parentNameUsageID").filter(pl.col("parentNameUsageID") == 2045)

speciesId,taxonName,parentNameUsageID,infraspecificEpithet,specificEpithet,…,family,order,class,phylum,kingdom
str,str,i64,str,str,…,str,str,str,str,str
"""A-Arth-Crus-Isopoda-000023""","""Sphaeromatidae sp. 1""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000024""","""Sphaeromatidae sp. 2""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000025""","""Sphaeromatidae sp. 3""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000026""","""Sphaeromatidae sp. 4""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000027""","""Sphaeromatidae sp. 5""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""A-Arth-Crus-Isopoda-000145""","""Sphaeromatidae sp. 44""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000146""","""Sphaeromatidae sp. 45""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""A-Arth-Crus-Isopoda-000147""","""Sphaeromatidae sp. 46""",2045,"""""","""""",…,"""Sphaeromatidae""","""Isopoda""","""Malacostraca""","""Arthropoda""","""Animalia"""


# matching for datapoints where phylum to genus ranks features are not empty strings

In [ ]:
_filter_genus = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") != "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
)

to_be_match_on_genus = still_no_match.filter(_filter_genus)

_x_genus = (
    still_no_match.filter(_filter_genus)
    .select("genus")
    .group_by("genus")
    .len()
)

_taxons_subset_genus = (
    taxons.filter(
        pl.col("genus").is_in(_x_genus["genus"].unique().to_list()),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
    )
    .select("taxonID", "genus")
    .collect()
)
match_on_genus = join_and_parentId_insertion(
    to_be_match_on_genus,
    _taxons_subset_genus,
    ["genus"],
)

In [ ]:
match_on_genus.select(
    ["speciesId", "taxonName", "parentNameUsageID"] + priority_columns
).sort("parentNameUsageID")

speciesId,taxonName,parentNameUsageID,infraspecificEpithet,specificEpithet,…,family,order,class,phylum,kingdom
str,str,i64,str,str,…,str,str,str,str,str
"""A-Bryozoa-000006""","""Celleporaria sp.""",1004617,"""""","""""",…,"""Lepraliellidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""
"""A-Bryozoa-000041""","""Celleporaria sp. 2""",1004617,"""""","""""",…,"""Lepraliellidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""
"""A-Bryozoa-000042""","""Celleporaria sp. 3""",1004617,"""""","""""",…,"""Lepraliellidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""
"""A-Bryozoa-000043""","""Celleporaria sp. 4""",1004617,"""""","""""",…,"""Lepraliellidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""
"""A-Bryozoa-000012""","""Triphyllozoon sp.""",1004892,"""""","""""",…,"""Phidoloporidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""A-Arth-Hexa-Diptera-002483""","""Siphona (Asiphona) sp. ZRC_BDP…",11592210,"""""","""""",…,"""Tachinidae""","""Diptera""","""Insecta""","""Arthopoda""","""Animalia"""
"""A-Arth-Hexa-Diptera-002613""","""Siphona (Asiphona) sp. ZRC_BDP…",11592210,"""""","""""",…,"""Tachinidae""","""Diptera""","""Insecta""","""Arthopoda""","""Animalia"""
"""A-Arth-Hexa-Diptera-002618""","""Siphona (Asiphona) sp. ZRC_BDP…",11592210,"""""","""""",…,"""Tachinidae""","""Diptera""","""Insecta""","""Arthopoda""","""Animalia"""


In [ ]:
match_on_genus.select(
    ["speciesId", "taxonName", "parentNameUsageID"] + priority_columns
).sort("parentNameUsageID").filter(pl.col("parentNameUsageID") == 1004617)

speciesId,taxonName,parentNameUsageID,infraspecificEpithet,specificEpithet,…,family,order,class,phylum,kingdom
str,str,i64,str,str,…,str,str,str,str,str
"""A-Bryozoa-000006""","""Celleporaria sp.""",1004617,"""""","""""",…,"""Lepraliellidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""
"""A-Bryozoa-000041""","""Celleporaria sp. 2""",1004617,"""""","""""",…,"""Lepraliellidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""
"""A-Bryozoa-000042""","""Celleporaria sp. 3""",1004617,"""""","""""",…,"""Lepraliellidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""
"""A-Bryozoa-000043""","""Celleporaria sp. 4""",1004617,"""""","""""",…,"""Lepraliellidae""","""Cheilostomatida""","""Gymnolaemata""","""Bryozoa""","""Animalia"""


In [ ]:
match_on_genus.select(
    ["speciesId", "taxonName", "parentNameUsageID"] + priority_columns
).filter(pl.col("taxonName") == "Paguristes sp. 1").sort("parentNameUsageID")

speciesId,taxonName,parentNameUsageID,infraspecificEpithet,specificEpithet,…,family,order,class,phylum,kingdom
str,str,i64,str,str,…,str,str,str,str,str
"""A-Arth-Crus-Decapoda-000285""","""Paguristes sp. 1""",2224457,"""""","""""",…,"""Diogenidae""","""Decapoda""","""Malacostraca""","""Arthropoda""","""Animalia"""


In [ ]:
match_on_genus.shape

## matching for datapoints where phylum to specificEpithet ranks features are not empty strings

In [ ]:
_filter_specificEpithet = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") != "",
    pl.col("specificEpithet") != "",
    pl.col("infraspecificEpithet") == "",
)

to_be_match_on_specificEpithet = still_no_match.filter(_filter_specificEpithet)

_x_specificEpithet = (
    still_no_match.filter(_filter_specificEpithet)
    .select("specificEpithet", "genus")
    .group_by("specificEpithet", "genus")
    .len()
)

_taxons_subset_specificEpithet = (
    taxons.filter(
        pl.col("specificEpithet").is_in(
            _x_specificEpithet["specificEpithet"].unique().to_list()
        ),
        pl.col("genus").is_in(_x_specificEpithet["genus"].unique().to_list()),
        pl.col("infraspecificEpithet").is_not_null(),
    )
    .select("taxonID", "specificEpithet", "genus")
    .collect()
)
match_on_specificEpithet = join_and_parentId_insertion(
    to_be_match_on_specificEpithet,
    _taxons_subset_specificEpithet,
    ["specificEpithet", "genus"],
)

In [ ]:
match_on_specificEpithet.select(
    "specificEpithet", "genus", "parentNameUsageID"
)

specificEpithet,genus,parentNameUsageID
str,str,i64


##matching for datapoints where phylum to infraspecificEpithet (Except for specificEpithet, no filters involved) ranks features are not empty strings.

In [ ]:
_filter_infraspecificEpithet = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") != "",
    pl.col("infraspecificEpithet") != "",
)  # different filter because some don't have specificEpithet either, if I am not wrong
to_be_match_on_infraspecificEpithet = still_no_match.filter(
    _filter_infraspecificEpithet
)

_x_infraspecificEpithet = (
    still_no_match.filter(_filter_infraspecificEpithet)
    .select("infraspecificEpithet", "genus")
    .group_by("infraspecificEpithet", "genus")
    .len()
)

_taxons_subset_infraspecificEpithet = (
    taxons.filter(
        pl.col("infraspecificEpithet").is_in(
            _x_infraspecificEpithet["infraspecificEpithet"].unique().to_list()
        ),
        pl.col("genus").is_in(
            _x_infraspecificEpithet["genus"].unique().to_list()
        ),
    )
    .select("taxonID", "infraspecificEpithet", "genus")
    .collect()
)

match_on_infraspecificEpithet = join_and_parentId_insertion(
    to_be_match_on_infraspecificEpithet,
    _taxons_subset_infraspecificEpithet,
    ["infraspecificEpithet", "genus"],
)

In [ ]:
match_on_infraspecificEpithet.select(
   ["speciesId", "taxonName", "parentNameUsageID"] + priority_columns 
).sort("parentNameUsageID")

speciesId,taxonName,parentNameUsageID,infraspecificEpithet,specificEpithet,…,family,order,class,phylum,kingdom
str,str,i64,str,str,…,str,str,str,str,str
"""A-Arth-Hexa-Lepidoptera-000056""","""Telicota besta bina""",6204724,"""bina""","""besta""",…,"""Hesperiidae""","""Lepidoptera""","""Insecta""","""""","""Animalia"""
"""A-Arth-Hexa-Lepidoptera-000243""","""Neptis leucoporos cresina""",6224905,"""cresina""","""leucoporos""",…,"""Nymphalidae""","""Lepidoptera""","""Insecta""","""""","""Animalia"""
"""A-Arth-Hexa-Lepidoptera-000192""","""Polyura schreiber tisamenus""",6233167,"""tisamenus""","""schreiber""",…,"""Nymphalidae""","""Lepidoptera""","""Insecta""","""""","""Animalia"""
"""A-Arth-Hexa-Lepidoptera-000162""","""Rapala suffusa barthema""",6236869,"""barthema""","""suffusa""",…,"""Lycaenidae""","""Lepidoptera""","""Insecta""","""""","""Animalia"""
"""A-Arth-Hexa-Lepidoptera-000442""","""Arhopala phanda phanda""",6237141,"""phanda""","""phanda""",…,"""Lycaenidae""","""Lepidoptera""","""Insecta""","""""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""A-Arth-Hexa-Lepidoptera-000326""","""Saletara panda distanti""",11488355,"""distanti""","""panda""",…,"""Pieridae""","""Lepidoptera""","""Insecta""","""""","""Animalia"""
"""A-Arth-Hexa-Coleoptera-000610""","""Taeniodera monacha modesta""",11747532,"""modesta""","""monacha""",…,"""Scarabaeidae""","""Coleoptera""","""Insecta""","""Arthropoda""","""Animalia"""
"""A-Arth-Hexa-Lepidoptera-000305""","""Troides amphyrysus ruficollis""",11764150,"""ruficollis""","""amphyrysus""",…,"""Papilionidae""","""Lepidoptera""","""Insecta""","""""","""Animalia"""


## Combining all these second matching wrangling stage dataframes of the ranks

In [ ]:
_to_drop = ["name_to_match"]
match_on_genus.write_csv("second_matches_set_for_genus_only.csv")
_new_match = pl.concat(
    [
        match_on_class.drop(_to_drop),
        match_on_order.drop(_to_drop),
        match_on_family.drop(_to_drop),
        match_on_genus,
        match_on_specificEpithet,
        match_on_infraspecificEpithet,
    ]
)
print("second wrangle new matches", _new_match.shape)
_new_match.write_csv("second_matches_set_from_wrangling.csv")
_new_match = pl.concat(
    [
        _new_match,
        updated_to_matching.with_columns(
            pl.col("parentNameUsageID").cast(pl.Int64)
        ),
    ]
)
_new_match.write_csv("updated_to_matching.csv")
_no_match = no_match.join(
    _new_match.select("speciesId"), on="speciesId", how="anti"
)
print("all matches", _new_match.shape)
print("no match", _no_match.shape)
_no_match.write_csv("no_match.csv")

second wrangle new matches (7177, 27)


all matches (7646, 27)
no match (1567, 27)


In [ ]:
bos.shape